In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [7]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [8]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,RENT,26000.0,Source Verified,Jan-2019,low_risk,n,9.60,...,80.0,0.0,0.0,0.0,20625.0,6798.0,11300.0,5425.0,N,N
68813,12000.0,0.2727,368.37,RENT,63000.0,Not Verified,Jan-2019,low_risk,n,29.07,...,96.2,0.0,0.0,0.0,87939.0,60350.0,13500.0,62939.0,N,N
68814,5000.0,0.1992,185.62,MORTGAGE,52000.0,Source Verified,Jan-2019,low_risk,n,14.86,...,100.0,0.0,1.0,0.0,30592.0,18611.0,3600.0,18492.0,N,N
68815,40000.0,0.0646,1225.24,MORTGAGE,520000.0,Verified,Jan-2019,low_risk,n,9.96,...,98.2,12.5,0.0,0.0,1033574.0,95958.0,100800.0,78634.0,N,N


In [9]:
# # Drop unnecessary columns
# df = df.drop(columns = [
#     "issue_d","dti", "delinq_2yrs", "inq_last_6mths",
#     "open_acc", "pub_rec", "revol_bal", "total_acc",
#     "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
#     "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
#     "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
#     "collections_12_mths_ex_med", "policy_code", "acc_now_delinq",
#     "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
#     "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
#     "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
#     "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
#     "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
#     "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
#     "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
#     "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
#     "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
#     "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
#     "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
#     "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75",
#     "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
#     "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"])
# df.head()

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_encoded = df.apply(le.fit_transform)
df_encoded

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,348,24,3500,3,2978,1,2,1,0,2590,...,285,155,0,0,11675,19018,19,27490,0,0
1,886,28,7846,1,4688,2,2,1,0,1889,...,331,101,1,0,39916,30368,412,21865,0,0
2,703,28,4973,1,2339,2,2,1,0,2292,...,142,101,0,0,10267,23239,75,18964,0,0
3,329,22,3251,3,4252,2,2,1,0,3010,...,409,101,1,0,19544,33595,197,32527,0,0
4,775,18,4894,1,2073,0,2,1,0,1742,...,409,0,0,0,34824,11407,276,6250,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,329,19,3166,3,495,1,1,1,0,826,...,240,0,0,0,777,1000,113,520,0,0
68813,404,37,3414,3,2792,0,1,1,0,2773,...,379,0,0,0,17147,30243,135,27854,0,0
68814,135,27,1289,1,2073,1,1,1,0,1352,...,409,0,1,0,2327,6840,35,5433,0,0
68815,1397,1,8649,1,6019,2,1,1,0,862,...,399,34,0,0,59325,41120,1010,33176,0,0


# Split the Data into Training and Testing

In [12]:
# Create our features
X = df_encoded.drop(columns=["loan_status"])

# Create our target
y = df_encoded["loan_status"]

In [13]:
y.describe()

count    68817.000000
mean         0.994958
std          0.070831
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: loan_status, dtype: float64

In [14]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [15]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [29]:
# # Creating a StandardScaler instance
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# #Fitting the SS with the training data
# X_scaler = scaler.fit(X_train)

# # Scaling the data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brf.fit(X_train, y_train)
y_pred = brf.predict(X_test)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7575192300567756

In [19]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)


[[   67    34]
 [ 2537 14567]]


In [22]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.03      0.66      0.85      0.05      0.75      0.55       101
          1       1.00      0.85      0.66      0.92      0.75      0.58     17104

avg / total       0.99      0.85      0.66      0.91      0.75      0.58     17205



In [36]:
# List the features sorted in descending order by feature importance
array = brf.feature_importances_

In [41]:
array = np.sort(array)[::-1]

In [42]:
array.tolist()

[0.08276522778252586,
 0.06902531799303391,
 0.05932897070449046,
 0.05911287492279406,
 0.046948094690217115,
 0.032897611982491665,
 0.024916668815835687,
 0.019023072471912258,
 0.01773450168080874,
 0.017320828152580153,
 0.016803750269332358,
 0.01612912197316762,
 0.01568013108887244,
 0.0153990393956703,
 0.015382647386255261,
 0.014864802629879959,
 0.014731903323056336,
 0.014656041758860369,
 0.014477443241605154,
 0.014297397766936563,
 0.014038380193810915,
 0.013652560598482189,
 0.013381679071619515,
 0.013306594881977284,
 0.013272863342321414,
 0.013140512004508989,
 0.012942787443140395,
 0.012703376474160496,
 0.012629411947611606,
 0.012629342696197218,
 0.011979554023834743,
 0.011766892430351611,
 0.010947317384398405,
 0.010907120322120588,
 0.010682899521940304,
 0.010542928422250424,
 0.010529531224126382,
 0.00993109195828378,
 0.009804672887687918,
 0.009537653550269467,
 0.009474206315866313,
 0.009264329697188779,
 0.008833762163294248,
 0.008798352023058077

In [ ]:
important_features_dict = {}
for x,i in enumerate(rf.feature_importances_):
    important_features_dict[x]=i


important_features_list = sorted(important_features_dict,
                                 key=important_features_dict.get,
                                 reverse=True)

print 'Most important features: %s' %important_features_list